In [1]:
import numpy as np
import pandas as pd

import warnings

warnings.filterwarnings("ignore")

In [7]:
# считываем данные
qid_query = pd.read_csv("../../PycharmProjects/RecSys_DEMO/Source/Yandex/qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("../../PycharmProjects/RecSys_DEMO/Source/Yandex/qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("../../PycharmProjects/RecSys_DEMO/Source/Yandex/hostid_url.tsv", sep="\t", names=["hostid", "url"])

# делаем join двух таблиц, чтобы было просто брать url с максимальным рейтингом
data = pd.merge(qid_url_rating, hostid_url, on="url")
data

,qid,url,rating,hostid
0,10387,http://batman-arkhamcity.ru/,0.00,64
1,10387,http://bigtorrents.org/publ/batman_arkham_city...,0.14,71
2,10387,http://consolelife.ru/xbox-360/6577-o-rossiysk...,0.14,101
3,10387,http://dic.academic.ru/book.nsf/3662736/Batman...,0.00,115
4,10387,http://forum.csmania.ru/viewtopic.php?t=25986,0.14,155
...,...,...,...,...
849,380923,http://www.spishy.ru/referats/14/8040,0.00,1063
850,380923,http://www.sprinter.ru/books/1821345.html,0.00,1065
851,380923,http://www.twirpx.com/files/mathematics/algebr...,0.00,1105
852,380923,http://wzyocgeawwo.eklablog.com/2-a60036325,0.00,1168


## Homework

### PFound
Исходные данные - Yandex Cup 2022 Analytics
- Ссылка - https://yandex.ru/cup/analytics/analysis/ , пример A. Рассчитать pFound
- Данные - https://yadi.sk/d/guqki4UI4hFlXQ
- Формула
$$pFound@K = \sum_{i=1}^{k} pLook[i]\ pRel[i]$$

$$pLook[1] = 1$$

$$pLook[i] = pLook[i-1]\ (1 - pRel[i-1])\ (1 - pBreak)$$

$$pBreak = 0.15$$

**Задача** - написать функцию, которая принимает на вход dataframe (после join), а на выходе дает средний pFound по всем query.
- Запрещается использовать циклы for для расчет метрики (как полностью, так и ее частей).
- Усложнение, если задача показалась легкой - попробуйте обойтись без groupby (не уверен, что это возможно, но вдруг вы справитесь)

In [20]:
%%timeit
def plook(ind, rels):
    if ind == 0:
                return 1
    return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)


def pfound(group):
    max_by_host = group.groupby("hostid")["rating"].max() # максимальный рейтинг хоста
    top10 = max_by_host.sort_values(ascending=False)[:10] # берем топ10 урлов с наивысшим рейтингом
    pfound = 0
    for ind, val in enumerate(top10):
                pfound += val*plook(ind, top10.values)
    return pfound


qid_pfound = data.groupby('qid', as_index=False).apply(pfound) # группируем по qid и вычисляем pfound

# Закомментируем остальной код образца для сопоставимого замера времени работы

# qid_pfound.rename(columns={None: 'pFound'}, inplace=True)
# qid_pfound
# qid_max = qid_pfound.idxmax() # берем qid с максимальным pfound

# qid_query[qid_query["qid"] == qid_max]

9.97 ms ± 7.09 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
qid_pfound

,qid,pFound
0,10387,0.497771
1,20860,0.655448
2,21070,0.497771
3,35618,0.437794
4,107538,0.354808
5,150126,0.366109
6,168170,0.481255
7,176370,0.393661
8,192007,0.191170
9,213932,0.347005


#### Векторизованный расчёт метрики pFound

In [32]:
def pFound_pandas(df: pd.DataFrame, query_col: str, rel_col: str) -> pd.DataFrame:
    df = df.sort_values(['qid', 'hostid', 'rating'], ascending=[True, True, False]).drop_duplicates(['qid', 'hostid']).reset_index(drop=True)
    df = df.sort_values(['qid', 'rating'], ascending=[True, False]).reset_index(drop=True)
    df['ind'] = df.groupby('qid').transform('cumcount')
    df = df[df.ind < 10]

    df['pLook'] = 1

    for i in range(1, 10):
        df[f'pLook_n'] = df[f'pLook'].shift(periods=1, fill_value=1) * (1 - df.rating.shift(periods=1, fill_value=1)) * 0.85
        df.loc[df.ind == i, 'pLook'] = df.loc[df.ind == i, 'pLook_n']

#     del df['pLook_n']

    df['pFound'] = df.rating * df.pLook
    df = df.groupby('qid', as_index=False).pFound.sum()

    return None #df   

In [33]:
%%timeit
pFound_pandas(data, 'qid', 'rating')

11.1 ms ± 77 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
pFound_pandas(data, 'qid', 'rating')

,qid,pFound
0,10387,0.497771
1,20860,0.655448
2,21070,0.497771
3,35618,0.437794
4,107538,0.354808
5,150126,0.366109
6,168170,0.481255
7,176370,0.393661
8,192007,0.191170
9,213932,0.347005
